In [2]:
from graph_ds import PyH3Graph
import h3.api.numpy_int as h3
import polars as pl

In [3]:
graph = PyH3Graph(weight_options={}, k_ring=2, layers="all")
graph.create(osm_path="../resources/copenhagen/copenhagen-processed.osm.pbf", gtfs_paths=["../resources/copenhagen/rejseplanen.zip"])

processing osm pbf file: ../resources/copenhagen/copenhagen-processed.osm.pbf
converted OSM file into 490098 edges
osm graph created with 418149 nodes in 2.4693465 s
getting GTFS feed from ../resources/copenhagen/rejseplanen.zip
routes: 1631
gtfs graph created with 103525 nodes in 9.260682 s
merged gtfs graph into osm graph, now has 519554 nodes, took 149 ms
hash: 8759942521107932551


In [3]:
# graph = PyH3Graph(weight_options=dict({
#     "bike_penalty": 1.0,
#     "wait_time_multiplier": 1.0,
#     "walk_speed": 1.4,
#     "bike_speed": 4.5
# }), k_ring=2, layers="all")
# graph.create(osm_path="../resources/copenhagen/copenhagen-processed.osm.pbf", gtfs_paths=["../resources/copenhagen/rejseplanen.zip"])

In [4]:
destinations = pl.read_csv("../resources/destinations/København Kommune_destinations_clean.csv")
destinations.head()

h3_index,category
i64,str
631049690136519679,"""library"""
631049690224658431,"""supermarket"""
631049690149925887,"""supermarket"""
631049690184231935,"""sport"""
631049684365186559,"""library"""


In [4]:
%%capture

def get_path(x):
    try:
        return graph.dijkstra_path(631049683665048575, x, None)
    except:
        return None

supermarket_path = get_path(h3.geo_to_h3(55.663076, 12.593622, 12))
school_path = get_path(h3.geo_to_h3(55.662401, 12.594020, 12))
pharmacy_path = get_path(h3.geo_to_h3(55.656520, 12.604307, 12))
sport_path = get_path(h3.geo_to_h3(55.660243, 12.592227, 12))
park_path = get_path(h3.geo_to_h3(55.660322, 12.588553, 12))
library_path = get_path(h3.geo_to_h3(55.662097, 12.589938, 12))

astar from 631049683665048575 to 631049683664541183
[backtrace] found start node
nodes have been adjusted: 631049683664425983 -> 631049683664427519
astar from 631049683665048575 to 631049683664427519
[backtrace] found start node
nodes have been adjusted: 631049683599036927 -> 631049683599053823
astar from 631049683665048575 to 631049683599053823
[backtrace] found start node
astar from 631049683665048575 to 631049683663521279
[backtrace] found start node
nodes have been adjusted: 631049683665037311 -> 631049683665114111
astar from 631049683665048575 to 631049683665114111
[backtrace] found start node
astar from 631049683665048575 to 631049683664582143
[backtrace] found start node


In [16]:
def path_to_xy(path):
    if path is None:
        return None
    return [h3.h3_to_geo(x) for x in path[0]]

arrays = []
destinations = []
labels = ["supermarket", "school", "pharmacy", "sport", "park", "library"]

for i, path in enumerate([supermarket_path, school_path, pharmacy_path, sport_path, park_path, library_path]):
    xy = path_to_xy(path)
    destinations.append([labels[i], xy[-1][0], xy[-1][1], " {:.2f}".format(path[1])])
    for coord in xy:
        arrays.append([labels[i], coord[0], coord[1], path[1]])

In [17]:
path_df = pl.DataFrame(arrays, columns=["type", "lat", "lon", "path_length"])
dest_df = pl.DataFrame(destinations, columns=["type", "lat", "lon", "path_length"])

/tmp/ipykernel_279416/2100517161.py:1: DeprecationWarning: `columns` is deprecated as an argument to `__init__`; use `schema` instead.
  path_df = pl.DataFrame(arrays, columns=["type", "lat", "lon", "path_length"])
/tmp/ipykernel_279416/2100517161.py:2: DeprecationWarning: `columns` is deprecated as an argument to `__init__`; use `schema` instead.
  dest_df = pl.DataFrame(destinations, columns=["type", "lat", "lon", "path_length"])


In [32]:
from lets_plot import *
from lets_plot import tilesets
LetsPlot.setup_html()

p = ggplot(path_df) + \
    geom_livemap(tiles=tilesets.LETS_PLOT_DARK) + \
    geom_path(aes('lon', 'lat', group='type', color='type'), size=2, alpha=0.5) + \
    geom_point(y=55.661110235731336, x=12.589319644730141, size=4) + \
    geom_point(aes('lon', 'lat', color='type'), data=dest_df, size=3, alpha=0.5) + \
    geom_text(aes('lon', 'lat', label='path_length', color='type'), data=dest_df, nudge_x=10, size=7) + \
    scale_color_discrete() + \
    ggsize(800, 600)

p.show()

In [31]:
h3.h3_to_geo(631049683665048575)

(55.661110235731336, 12.589319644730141)